<a href="https://colab.research.google.com/github/compi1234/pyspch/blob/master/test/audio_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mini Recorder Test Notebook 


Date: 17/05/2021

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

from IPython.display import display, clear_output, Audio, HTML
import ipywidgets as widgets
import numpy as np


import librosa
try:
  import google.colab
  IN_COLAB = True 
  ! pip install git+https://github.com/compi1234/pyspch.git
except:
  IN_COLAB = False

# verify the IPython version
import IPython
if IPython.version_info[0] >= 6:
    Audio_args = {'normalize':False}
else:
    print("Warning: you are using IPython<6 \n IPython.display.Audio() will automatically normalize audio output")
    Audio_args = {}
import pyspch.audio as audio
import pyspch.spectrogram as specg
import pyspch.utils as spch_utils
import pyspch.display as spch_disp
#import pyspch.display_ly as spch_disp
import pyspch

SAMPLE_WAV_URL = 'https://homes.esat.kuleuven.be/~spchlab/data/misc/train.wav'
SAMPLE_WAV_13dB_SNGER = 'https://homes.esat.kuleuven.be/~spchlab/data/13dB/Psnger22.wav'

In [2]:
Symbols = { 'play':'\u25b6','reverse':'\u25C0' , 'pause':'\u23F8', 'stop': '\u23F9', 'record':'\u2b55'}
def box_layout():
     return widgets.Layout(
        border='solid 1px black',
        margin='0px 10px 10px 0px',
        padding='5px 5px 5px 5px'
     )
def button_layout():
    return widgets.Layout(
        border='solid 1px black',
        margin='0px 10px 10px 0px',
        padding='5px 5px 5px 5px',
        width = '40px',
        height = '40px',
        flex_shrink =2
     )

class mini_recorder(widgets.VBox):
    
    def __init__(self,data=np.zeros((1000,)),sample_rate=16000,figsize=(5,4),dpi=100):
        super().__init__()
        
        self.data = data
        
        self.sample_rate = sample_rate
        self.rec_time = 2.0         
        self.line_color = '#0000ff'
        self.figsize = figsize
        self.dpi = dpi
                     
        self.wg_play_button = widgets.Button(description=Symbols['play'],layout=button_layout())
        self.wg_record_button = widgets.Button(description=Symbols['record'],layout=button_layout())
        self.wg_pause_button = widgets.Button(description=Symbols['pause'],layout=button_layout())
        self.wg_clear_log_button = widgets.Button(description='Clear log')
        #self.rec_time_slider = widgets.FloatSlider(value=self.rec_time,step=.5,min=.5,max=10.,
        #                                           description='Rec Time',indent=False,continous_update=False)

        self.wg_rectime = widgets.BoundedFloatText(   value=2.0, min=0.5, max= 10., step=0.5,
            description='Rec Time:', disabled=False)
        self.out = widgets.Output(layout=box_layout())
        self.out.layout.width = '100%'
        self.logscr = widgets.Output()        
        self.UI = widgets.VBox([
            widgets.HBox([self.wg_play_button,self.wg_record_button,self.wg_pause_button,self.wg_rectime,self.wg_clear_log_button]),
            self.logscr], 
            layout=box_layout())
        self.UI.layout.width = '100%'
        # add as children
        self.children = [self.out, self.UI] 
        
        self.wg_play_button.on_click(self.play_sound)       
        self.wg_record_button.on_click(self.record_sound)
        self.wg_pause_button.on_click(self.pause_sound)
        self.wg_clear_log_button.on_click(self.clear_log)
        self.wg_rectime.observe(self.rectime_observe,'value')
        
        self.plot_data()
        plt.close()


    def plot_data(self):
        with self.out:
            clear_output(wait=True)
            spg = specg.spectrogram(self.data,sample_rate=self.sample_rate)
            self.fig = spch_disp.plot_spg(spg,wav=self.data,sample_rate=self.sample_rate,figsize=self.figsize,dpi=self.dpi)
            #self.fig = spch_disp.plot_waveform(self.data,sample_rate=self.sample_rate)
            display(self.fig)
        
    def rectime_observe(self,change):
        self.rec_time = change.new
    
    def pause_sound(self,b):
        with self.logscr:
            print("You didn't expect everything to work ?!! did you ... ")
            
    def play_sound(self,b):
        with self.logscr:
            clear_output()
            if(IN_COLAB):
              print("IN_COLAB: Use the HTML button to play sound")
            audio.play(self.data,sample_rate=self.sample_rate,wait=True)
        
    def record_sound(self,b):      
        with self.logscr:
            self.data = audio.record(self.rec_time,self.sample_rate,n_channels=1)
        self.plot_data()
        self.play_sound(b)
     
    def clear_log(self,b):
        with self.logscr: clear_output()
            


In [3]:
recorder = mini_recorder(figsize=(8,6))
recorder

mini_recorder(children=(Output(layout=Layout(border='solid 1px black', margin='0px 10px 10px 0px', padding='5p…